In [2]:
using Pkg
Pkg.activate("psyc261")
using Distributions
using ProgressMeter
using Gen, Plots
using Parameters
using PyCall

ENV["PYTHON"] = "/Users/dongyu/opt/anaconda3/envs/modeling/bin/python"
println(PyCall.python)

  Activating project at `~/Documents/Github/final_project/lab-05/psyc261`


/Users/dongyu/opt/anaconda3/envs/modeling/bin/python


In [3]:
np = pyimport("numpy")
mi = pyimport("mitsuba")

PyObject <module 'mitsuba'>

In [14]:
py"""
import numpy as np
import mitsuba as mi
mi.set_variant("scalar_rgb")
from mitsuba import ScalarTransform4f as T
def initialize_scene(leftwall_color, rightwall_color, leftobject_type, rightobject_type):

    d = {
        "type": "scene",
        "integrator": {
            'type': 'path',
            'max_depth': 6,
        },
        
        "sensor": {
            'type': 'perspective',
            'fov_axis': 'smaller',
            'near_clip': 0.001,
            'far_clip': 100.0,
            'focus_distance': 1000,
            'fov': 39.3077,
            'to_world': T.look_at(
                origin=[0, 0, 4],
                target=[0, 0, 0],
                up=[0, 1, 0]
            ),
            'sampler': {
                'type': 'independent',
                'sample_count': 16
            },
            'film': {
                'type': 'hdrfilm',
                'width' : 64,
                'height': 64,
                'rfilter': {
                    'type': 'tent',
                },
                'pixel_format': 'rgb',
                'component_format': 'float32',
            }
        },

        # -------------------- BSDFs --------------------
        "gray": {
            'type': 'diffuse',
            'reflectance': {
                'type': 'rgb',
                'value': [0.85, 0.85, 0.85],
            }
        },

        "color-sphere": {
            'type': 'twosided',
            'material': {
                'type': 'diffuse',
                    'reflectance': {
                    'type': 'rgb',
                    'value': [0.725, 0.71, 0.68],
                }
            }
        },
 
 
        "white": {
            'type': 'diffuse',
            'reflectance': {
                'type': 'rgb',
                'value': [0.885809, 0.698859, 0.666422],
            }
        },

        "green": {
            'type': 'diffuse',
            'reflectance': {
                'type': 'rgb',
                'value': [0.105421, 0.37798, 0.076425],
            }
        },
        
        "red": {
            'type': 'diffuse',
            'reflectance': {
                'type': 'rgb',
                'value': [0.570068, 0.0430135, 0.0443706],
            }
        },
        
        # -------------------- Light --------------------
        "light": {
            'type': 'rectangle',
            'to_world': T.translate([0.0, 0.99, 0.01]).rotate([1, 0, 0], 90).scale([0.23, 0.19, 0.19]),
            'bsdf': {
                'type': 'ref',
                'id': 'white'
            },
            'emitter': {
                'type': 'area',
                'radiance': {
                    'type': 'rgb',
                    'value': [18.387, 13.9873, 6.75357],
                }
            }
        },

        "floor": {
            'type': 'obj',
            'filename': 'meshes/cbox_floor.obj',
            'floor-bsdf': {
                'type' : 'ref',
                'id' : 'white'
            }
        },

        "ceiling": {
            'type': 'obj',
            'filename': 'meshes/cbox_ceiling.obj',
            'ceiling-bsdf': {
                'type' : 'ref',
                'id' : 'white'
            }
        },
        
        "back": {
            'type': 'obj',
            'filename': 'meshes/cbox_back.obj',
            'back-bsdf': {
                'type' : 'ref',
                'id' : 'white'
            }
        },

        "leftwall": {
            'type': 'obj',
            'filename': 'meshes/cbox_greenwall.obj',
            'greenwall-bsdf': {
                'type' : 'ref',
                'id' : leftwall_color
            }
        },

        "rightwall": {
            'type': 'obj',
            'filename': 'meshes/cbox_redwall.obj',
            'redwall-bsdf': {
                'type' : 'ref',
                'id' : rightwall_color
            }
        },
        
        "left-object": {
            'type': leftobject_type,
            'to_world': T.translate([-0.3, -0.5, 0.2]).scale(0.5),
            'leftobject-bsdf': {
                'type' : 'ref',
                'id' : "color-sphere"
            }
        },

        "right-object": {
            'type': rightobject_type,
            'to_world': T.translate([0.5, -0.75, -0.2]).scale(0.5),
            'rightobject-bsdf': {
                'type' : 'ref',
                'id' : "color-sphere"
            }
        }

    }
    return d

"""

In [16]:
leftwall_colors = ["white", "green", "red"]
rightwall_colors = ["white", "green", "red"]
leftobject_types = ["sphere", "rectangle", "slab"]
rightobject_types = ["sphere", "rectangle", "slab"]
spp = 128


128

In [17]:

for (lw_color, rw_color, lo_type, ro_type) in Iterators.product(leftwall_colors, rightwall_colors, leftobject_types, rightobject_types)
    scene_d = py"initialize_scene"(lw_color, rw_color, lo_type, ro_type)
    # load the scene using the load_dict function
    scene = mi.load_dict(scene_d)
    # render the scene
    image = mi.render(scene, spp=spp)
    # get the bitmap, which Jupyter notebook displays inline!
    bitmap = mi.Bitmap(image).convert(srgb_gamma=true)
end

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /Users/dongyu/.julia/packages/PyCall/KLzIO/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError('\u200b[xml_v.cpp:353] Referenced id "red" not found: leftwall')


In [ ]:
@gen function room()

    # load the scene configuration, including a room with colored walls, two spheres, lights, materials, and a sensor
    scene_d = py"initialize_scene()"o::PyObject
    
    # prior over the scale of the left sphere
    s_left ~ uniform(0.1, 1.0)
    # we assume we know a priori where this sphere will appear in the scene
    t_left = [-0.3, -0.5, 0.2]
    # transform the object in the scene according to this scale and translation parameters
    transform!(scene_d, "left-object", s_left, t_left)
    
    # prior over the scale of the right sphere
    s_right ~ uniform(0.1, 1.0)
    # we assume we know a priori where this sphere will appear in the scene
    t_right = [0.5, -0.75, -0.2] 
    # transform the object in the scene according to this scale and translation parameters
    transform!(scene_d, "right-object", s_right, t_right)

    # load the resulting scene into our struct
    # Notice that we are using a much cheaper spp for inference
    modelparams = ModelParams(scene_d=scene_d, spp=16)

    # render into an array
    mu = render(modelparams)

    # define the likelihood as a gaussian with diagonal covariance
    pred ~ broadcasted_normal(mu, 1.0)

end
;

In [ ]:
function view_image(t)
    scene_d = py"initialize_scene()"o::PyObject
    t_left = [-0.3, -0.5, 0.2]
    transform!(scene_d, "left-object", t[:s_left], t_left)
    
    t_right = [0.5, -0.75, -0.2] 
    transform!(scene_d, "right-object", t[:s_right], t_right)
    #scene = mi.load_dict(scene_d)
    scene_to_render = ModelParams(scene_d=scene_d)
    bitmap = render_bitmap(scene_to_render)
end
;